<a href="https://www.kaggle.com/code/adityavashist8215/dog-vs-cat-classifier?scriptVersionId=247851532" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D,Dense,BatchNormalization,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

# Transfer Learning

In [21]:
# creating an object of vgg16
conv_base=VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(256,256,3),
)

In [22]:
conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 256, 256, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 256, 256, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 128, 128, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 128, 128, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 16, 16, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 8, 8, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model=Sequential([
    conv_base,
    Flatten(),

    Dense(128,activation='relu',kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64,activation='relu',kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.1),
    Dense(1,activation='sigmoid'),
])

In [24]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 8, 8, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │       4,194,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,918,209 (72.17 MB)

 Trainable params: 18,917,825 (72.17 MB)

 Non-trainable params: 384 (1.50 KB)

In [25]:
# Freezing the trainable parameters
conv_base.trainable=False

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [27]:
batch_size=16

# object creation for train and test
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2, 
    horizontal_flip=True,
    
)

test_datagen=ImageDataGenerator(rescale=1./255)

In [28]:
train_generator=train_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats/train',
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary'
)
test_generator=test_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats/test',
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [29]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)

In [30]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping

# Define warm-up scheduler function
def warmup_scheduler(epoch):
    if epoch < 3:  # First 3 epochs = warm-up
        return 1e-4 + (epoch * 2e-4)  # Gradually increase LR
    else:
        return 0.001  # After warm-up, use stable learning rate

# Add to callback
lr_callback = LearningRateScheduler(warmup_scheduler)

In [32]:
callback=EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00001,
    patience=10,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
)

In [33]:
history=model.fit(train_generator,epochs=30,validation_data=test_generator,steps_per_epoch=2000,validation_steps=800,callbacks=[lr_callback,callback])

Epoch 1/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 377s 186ms/step - accuracy: 0.8370 - loss: 0.3945 - val_accuracy: 0.9362 - val_loss: 0.1978 - learning_rate: 1.0000e-04
Epoch 2/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 293s 146ms/step - accuracy: 0.8842 - loss: 0.3081 - val_accuracy: 0.9234 - val_loss: 0.2143 - learning_rate: 3.0000e-04
Epoch 3/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 293s 146ms/step - accuracy: 0.8877 - loss: 0.3001 - val_accuracy: 0.9290 - val_loss: 0.2055 - learning_rate: 5.0000e-04
Epoch 4/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 290s 145ms/step - accuracy: 0.8895 - loss: 0.3064 - val_accuracy: 0.9340 - val_loss: 0.2178 - learning_rate: 0.0010
Epoch 5/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 294s 147ms/step - accuracy: 0.8962 - loss: 0.3053 - val_accuracy: 0.9130 - val_loss: 0.2629 - learning_rate: 0.0010
Epoch 6/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 288s 144ms/step - accuracy: 0.8945 - loss: 0.3191 - val_accuracy: 0.8928 - val_loss: 0.3222 - learning_rate: 0.0010
Epoch 7/30
2000/2000 ━━━━━━━━━━━━━━━━━━━

# Here we can give each image to the model with their respective labels but because images are in a large numbers, SO keras uses the concept of generators
# generators - process large amount of data
# 2 generators- for test and train

In [246]:
# train_ds=keras.utils.image_dataset_from_directory(
#     directory = "/kaggle/input/dogs-vs-cats/train",
#     labels = 'inferred',
#     label_mode ='int',
#     batch_size=32,
#     image_size=(256,256)
    
# )
# test_ds=keras.utils.image_dataset_from_directory(
#     directory = "/kaggle/input/dogs-vs-cats/test",
#     labels = 'inferred',
#     label_mode ='int',
#     batch_size=32,
#     image_size=(256,256)
    
# )


In [247]:
# # Normalization 255 to (0-1)
# def process(image,label):
#   image=tf.cast(image/255. , tf.float32)
#   return image,label
# train_ds=train_ds.map(process)
# test_ds=test_ds.map(process)

In [248]:
batch_size=16

# object creation for train and test
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2, 
    horizontal_flip=True,
    
)

test_datagen=ImageDataGenerator(rescale=1./255)

In [249]:
train_generator=train_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats/train',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)
test_generator=test_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats/test',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


# Create CNN Model

In [250]:
model=Sequential([
    Conv2D(64,kernel_size=(3,3),kernel_regularizer=l2(1e-4),activation='relu',padding='valid',input_shape=(150,150,3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),

    Conv2D(128,kernel_size=(3,3),kernel_regularizer=l2(1e-4),activation='relu',padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),

    Conv2D(256,kernel_size=(3,3),kernel_regularizer=l2(1e-4),activation='relu',padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),

    Flatten(),

    Dense(128,activation='relu',kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64,activation='relu',kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dense(1,activation='sigmoid'),
])


In [251]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_45 (Conv2D)                   │ (None, 148, 148, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_75               │ (None, 148, 148, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_45 (MaxPooling2D)      │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_46 (Conv2D)                   │ (None, 72, 72, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_76               │ (None, 72, 72, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_46 (MaxPooling2D)      │ (None, 36, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_47 (Conv2D)                   │ (None, 34, 34, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_77               │ (None, 34, 34, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_47 (MaxPooling2D)      │ (None, 17, 17, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_15 (Flatten)                 │ (None, 73984)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 128)                 │       9,470,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_78               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_79               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,851,777 (37.58 MB)

 Trainable params: 9,850,497 (37.58 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [252]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)

In [253]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [254]:
print(type(model))


<class 'keras.src.models.sequential.Sequential'>


In [255]:
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping

# Define warm-up scheduler function
def warmup_scheduler(epoch):
    if epoch < 3:  # First 3 epochs = warm-up
        return 1e-4 + (epoch * 2e-4)  # Gradually increase LR
    else:
        return 0.001  # After warm-up, use stable learning rate

# Add to callback
lr_callback = LearningRateScheduler(warmup_scheduler)


In [256]:
callback=EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.0001,
    patience=10,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
)

In [257]:
history=model.fit(train_generator,epochs=25,validation_data=test_generator,steps_per_epoch=2000,validation_steps=800,callbacks=[lr_callback,callback])

Epoch 1/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 139s 65ms/step - accuracy: 0.6417 - loss: 0.7369 - val_accuracy: 0.7246 - val_loss: 0.6065 - learning_rate: 1.0000e-04
Epoch 2/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 131s 65ms/step - accuracy: 0.6968 - loss: 0.6567 - val_accuracy: 0.6960 - val_loss: 0.7038 - learning_rate: 3.0000e-04
Epoch 3/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 131s 66ms/step - accuracy: 0.7455 - loss: 0.6158 - val_accuracy: 0.7466 - val_loss: 0.6170 - learning_rate: 5.0000e-04
Epoch 4/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 133s 66ms/step - accuracy: 0.7652 - loss: 0.6365 - val_accuracy: 0.7456 - val_loss: 0.6761 - learning_rate: 0.0010
Epoch 5/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 135s 67ms/step - accuracy: 0.7889 - loss: 0.6476 - val_accuracy: 0.7832 - val_loss: 0.6956 - learning_rate: 0.0010
Epoch 6/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 130s 65ms/step - accuracy: 0.8082 - loss: 0.6932 - val_accuracy: 0.7206 - val_loss: 0.8889 - learning_rate: 0.0010
Epoch 7/25
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 130s

In [261]:
history = model.fit(
    train_generator,
    epochs=30,  # total epochs you want to train for (e.g., from 25 to 50)
    initial_epoch=25,  # start from epoch 25
    validation_data=test_generator,
    steps_per_epoch=2000,
    validation_steps=800,
    callbacks=[lr_callback, callback]
)

Epoch 26/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 152s 76ms/step - accuracy: 0.9037 - loss: 0.5147 - val_accuracy: 0.8960 - val_loss: 0.5220 - learning_rate: 0.0010
Epoch 27/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 127s 64ms/step - accuracy: 0.9052 - loss: 0.5141 - val_accuracy: 0.8424 - val_loss: 0.7051 - learning_rate: 0.0010
Epoch 28/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 128s 64ms/step - accuracy: 0.9043 - loss: 0.5030 - val_accuracy: 0.8954 - val_loss: 0.5298 - learning_rate: 0.0010
Epoch 29/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 129s 64ms/step - accuracy: 0.9017 - loss: 0.5076 - val_accuracy: 0.9060 - val_loss: 0.4956 - learning_rate: 0.0010
Epoch 30/30
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 128s 64ms/step - accuracy: 0.9031 - loss: 0.5057 - val_accuracy: 0.9148 - val_loss: 0.4833 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 30.


In [258]:
# history=model.fit(train_ds,epochs=30,validation_data=test_ds,verbose=1,callbacks=[lr_callback,callback])

In [ ]:
history = model.fit(
    train_generator,
    epochs=40,  # total epochs you want to train for (e.g., from 25 to 50)
    initial_epoch=30,  # start from epoch 25
    validation_data=test_generator,
    steps_per_epoch=2000,
    validation_steps=800,
    callbacks=[lr_callback, callback]
)

Epoch 31/40
 298/2000 ━━━━━━━━━━━━━━━━━━━━ 3:08 111ms/step - accuracy: 0.9035 - loss: 0.5002

After Data Augmentation -val_accuracy-92.12

<!-- model=Sequential([
    Conv2D(32,kernel_size=(3,3),kernel_regularizer=l2(1e-4),activation='relu',padding='valid',input_shape=(256,256,3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),

    Conv2D(64,kernel_size=(3,3),kernel_regularizer=l2(1e-4),activation='relu',padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),

    Conv2D(128,kernel_size=(3,3),kernel_regularizer=l2(1e-4),activation='relu',padding='valid'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),

    Flatten(),

    Dense(128,activation='relu',kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64,activation='relu',kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dropout(0.1),
    Dense(1,activation='sigmoid'),
]) -->

 val_acc=86.14